# Testing a new environment

https://stackoverflow.com/questions/45068568/is-it-possible-to-create-a-new-gym-environment-in-openai

## How to create new environments for Gym

* Create a new repo called gym-foo, which should also be a PIP package.

* A good example is https://github.com/openai/gym-soccer.

* It should have at least the following files:
  ```sh
  gym-foo/
    README.md
    setup.py
    gym_foo/
      __init__.py
      envs/
        __init__.py
        foo_env.py
        foo_extrahard_env.py
  ```

* `gym-foo/setup.py` should have:

  ```python
  from setuptools import setup

  setup(name='gym_foo',
        version='0.0.1',
        install_requires=['gym']  # And any other dependencies foo needs
  )  
  ```

* `gym-foo/gym_foo/__init__.py` should have:
  ```python
  from gym.envs.registration import register

  register(
      id='foo-v0',
      entry_point='gym_foo.envs:FooEnv',
  )
  register(
      id='foo-extrahard-v0',
      entry_point='gym_foo.envs:FooExtraHardEnv',
  )
  ```

* `gym-foo/gym_foo/envs/__init__.py` should have:
  ```python
  from gym_foo.envs.foo_env import FooEnv
  from gym_foo.envs.foo_extrahard_env import FooExtraHardEnv
  ```

* `gym-foo/gym_foo/envs/foo_env.py` should look something like:
  ```python
  import gym
  from gym import error, spaces, utils
  from gym.utils import seeding

  class FooEnv(gym.Env):
    metadata = {'render.modes': ['human']}

    def __init__(self):
      ...
    def _step(self, action):
      ...
    def _reset(self):
      ...
    def _render(self, mode='human', close=False):
      ...
  ```


In [1]:
from gym.envs.registration import register
import gym
from gym import error, spaces, utils
from gym.utils import seeding
import numpy as np


class FooEnv(gym.Env):
    metadata = {'render.modes': ['text']}

    def __init__(self):
        high = np.array([3, 4, 5])  # TODO: example value only
        self.action_space = spaces.Discrete(2)
        self.observation_space = spaces.Box(-high, high)
        self._seed()
        self.state = None

    def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def _step(self, action):
        assert self.action_space.contains(action), "%r (%s) invalid"%(action, type(action))
        state = self.state

    def _reset(self):
        self.state = self.np_random.uniform(low=-0.05, high=0.05, size=(4,))
        self.steps_beyond_done = None
        return np.array(self.state)

    def _render(self, mode='text', close=False):
        assert mode == 'text' or close
        if mode == 'text':
            print(self.state)

register(
    id='foo-v0',
    entry_point=FooEnv,
)

In [6]:
env = gym.make('foo-v0')
print(env.action_space)
print(env.observation_space)
print(env.reward_range)
print(env.reset())
print(env.action_space.sample())

help(env.action_space)


Discrete(2)
Box(3,)
(-inf, inf)
[-0.048292    0.0413046  -0.00485422 -0.02624112]
1
Help on Discrete in module gym.spaces.discrete object:

class Discrete(gym.core.Space)
 |  {0,1,...,n-1}
 |  
 |  Example usage:
 |  self.observation_space = spaces.Discrete(2)
 |  
 |  Method resolution order:
 |      Discrete
 |      gym.core.Space
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __eq__(self, other)
 |      Return self==value.
 |  
 |  __init__(self, n)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  contains(self, x)
 |      Return boolean specifying if x is a valid
 |      member of this space
 |  
 |  sample(self)
 |      Uniformly randomly sample a random element of this space
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  shape
 |  
 |  ----------------------------------------------------------------------
 